In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
import numpy as np
import torch
import cv2
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
data_path_500 = '/hdd/sd8/tlc/TCT/all_data/500_neg_416/'
data_path_270 = '/hdd/sd8/tlc/TCT/all_data/20190717/'
excel_label = '/home/yuyue/yuyue/MIL_TCT/MIL_pytorch/excel/500_label.csv'
negative_uid = '/hdd/sd2/data/TCT/densenet/data/500_data/val_500_neg.pth'

In [6]:
def inference(image):
    with tf.Graph().as_default() as graph:
        with tf.Session(graph=graph) as sess:
            tf.global_variables_initializer().run()
            with gfile.FastGFile('/hdd/sd8/tlc/TCT/Model_pb/yolov3_test_loss=13.8747.ckpt.pb', 'rb') as f:
                graph_def = tf.GraphDef()
                graph_def.ParseFromString(f.read())
                tf.import_graph_def(graph_def, name='') # 导入计算图
                input_ = sess.graph.get_tensor_by_name('input_data:0')
                #trainable = sess.graph.get_tensor_by_name('training:0')
                op_sbbox = sess.graph.get_tensor_by_name('pred_sbbox/op_to_store:0')
                op_mbbox = sess.graph.get_tensor_by_name('pred_mbbox/op_to_store:0')
                op_lbbox = sess.graph.get_tensor_by_name('pred_lbbox/op_to_store:0')
                ret_sbbox, ret_mbbox, ret_lbbox= sess.run([op_sbbox,op_mbbox,op_lbbox], feed_dict={input_:image})
                return ret_sbbox, ret_mbbox, ret_lbbox
            
def img_resize(image, target_size):
    ih, iw    = target_size
    h,  w, _  = image.shape

    scale = min(float(iw)/w, float(ih)/h)
    nw, nh  = int(scale * w), int(scale * h)
    try:
        image_resized = cv2.resize(image, (nw, nh), interpolation=1)
    except Exception as e:
        print (str(e))
        
    image_paded = np.full(shape=[ih, iw, 3], fill_value=128.0)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2
    image_paded[dh:nh+dh, dw:nw+dw, :] = image_resized
    return image_paded

def postprocess_boxes(pred_bbox, org_img_shape, input_size, score_threshold):
    valid_scale=[0, np.inf]
    
    pred_bbox = np.array(pred_bbox)
    #print (pred_bbox.shape)
    pred_xywh = pred_bbox[:, 0:4]
    pred_conf = pred_bbox[:, 4]
    pred_prob = pred_bbox[:, 5:]
    # # (1) (x, y, w, h) --> (xmin, ymin, xmax, ymax)
    pred_coor = np.concatenate([pred_xywh[:, :2] - pred_xywh[:, 2:] * 0.5,
                                pred_xywh[:, :2] + pred_xywh[:, 2:] * 0.5], axis=-1)
    # # (2) (xmin, ymin, xmax, ymax) -> (xmin_org, ymin_org, xmax_org, ymax_org)
    org_h, org_w = org_img_shape
    resize_ratio = min(float(input_size) / org_w, float(input_size) / org_h)

    dw = (input_size - resize_ratio * org_w) / 2
    dh = (input_size - resize_ratio * org_h) / 2

    pred_coor[:, 0::2] = 1.0 * (pred_coor[:, 0::2] - dw) / resize_ratio
    pred_coor[:, 1::2] = 1.0 * (pred_coor[:, 1::2] - dh) / resize_ratio

    # # (3) clip some boxes those are out of range
    pred_coor = np.concatenate([np.maximum(pred_coor[:, :2], [0, 0]),
                                np.minimum(pred_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
    invalid_mask = np.logical_or((pred_coor[:, 0] > pred_coor[:, 2]), (pred_coor[:, 1] > pred_coor[:, 3]))
    pred_coor[invalid_mask] = 0
    

    # # (4) discard some invalid boxes
    bboxes_scale = np.sqrt(np.multiply.reduce(pred_coor[:, 2:4] - pred_coor[:, 0:2], axis=-1))
    scale_mask = np.logical_and((valid_scale[0] < bboxes_scale), (bboxes_scale < valid_scale[1]))
    #print np.where(scale_mask is True)

    # # (5) discard some boxes with low scores
    classes = np.argmax(pred_prob, axis=-1)
    scores = pred_conf * pred_prob[np.arange(len(pred_coor)), classes]
    #print (np.max(scores))
    score_mask = scores > score_threshold
    #print np.sum(score_mask)
    mask = np.logical_and(scale_mask, score_mask)
    coors, scores, classes = pred_coor[mask], scores[mask], classes[mask]

    return np.concatenate([coors, scores[:, np.newaxis], classes[:, np.newaxis]], axis=-1)

def bboxes_iou(boxes1, boxes2):

    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)

    boxes1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    boxes2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])

    left_up       = np.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down    = np.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = np.maximum(right_down - left_up, 0.0)
    inter_area    = inter_section[..., 0] * inter_section[..., 1]
    union_area    = boxes1_area + boxes2_area - inter_area
    ious          = np.maximum(1.0 * inter_area / union_area, np.finfo(np.float32).eps)

    return ious

def nms(bboxes, iou_threshold, sigma=0.3, method='nms'):
    """
    :param bboxes: (xmin, ymin, xmax, ymax, score, class)

    Note: soft-nms, https://arxiv.org/pdf/1704.04503.pdf
          https://github.com/bharatsingh430/soft-nms
    """
    classes_in_img = list(set(bboxes[:, 5]))
    best_bboxes = []

    for cls in classes_in_img:
        cls_mask = (bboxes[:, 5] == cls)
        cls_bboxes = bboxes[cls_mask]

        while len(cls_bboxes) > 0:
            max_ind = np.argmax(cls_bboxes[:, 4])
            best_bbox = cls_bboxes[max_ind]
            best_bboxes.append(best_bbox)
            cls_bboxes = np.concatenate([cls_bboxes[: max_ind], cls_bboxes[max_ind + 1:]])
            iou = bboxes_iou(best_bbox[np.newaxis, :4], cls_bboxes[:, :4])
            weight = np.ones((len(iou),), dtype=np.float32)

            assert method in ['nms', 'soft-nms']
            if method == 'nms':
                iou_mask = iou > iou_threshold
                weight[iou_mask] = 0.0

            if method == 'soft-nms':
                weight = np.exp(-(1.0 * iou ** 2 / sigma))

            cls_bboxes[:, 4] = cls_bboxes[:, 4] * weight
            score_mask = cls_bboxes[:, 4] > 0.
            cls_bboxes = cls_bboxes[score_mask]

    return best_bboxes

In [7]:
uid_list = torch.load(negative_uid)
map_type = {}
with tf.Graph().as_default() as graph:
    with tf.Session(graph=graph) as sess:
        tf.global_variables_initializer().run()
        with gfile.FastGFile('/hdd/sd8/tlc/TCT/Model_pb/yolov3_test_loss=13.8747.ckpt.pb', 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='') # 导入计算图
            input_ = sess.graph.get_tensor_by_name('input_data:0')
            #trainable = sess.graph.get_tensor_by_name('training:0')
            op_sbbox = sess.graph.get_tensor_by_name('pred_sbbox/op_to_store:0')
            op_mbbox = sess.graph.get_tensor_by_name('pred_mbbox/op_to_store:0')
            op_lbbox = sess.graph.get_tensor_by_name('pred_lbbox/op_to_store:0')
            for uid in tqdm_notebook(uid_list):
                folder_path = os.path.join(data_path_500, uid)
                for img in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, img)
                    image = cv2.imread(img_path)
                    #image = img_resize(ori_image, (416,416))
                    image = image/255.
                    input_image = image[np.newaxis, :, :, :]
                    pred_sbbox, pred_mbbox, pred_lbbox= sess.run([op_sbbox,op_mbbox,op_lbbox], feed_dict={input_:input_image})
                    pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + 8)),
                                                 np.reshape(pred_mbbox, (-1, 5 + 8)),
                                                 np.reshape(pred_lbbox, (-1, 5 + 8))], axis=0)

                    bboxes = postprocess_boxes(pred_bbox, (ori_image.shape[0], ori_image.shape[1]), 416, 0.7)
                    bboxes = nms(bboxes, 0.3)
                    for bbox in bboxes:
                        coor = np.array(bbox[:4], dtype=np.int32)
                        score = bbox[4]
                        type_ = bbox[5]
                        if type_ not in map_type:
                            map_type[type_] = 1
                        else:
                            map_type[type_] += 1
            

In [8]:
map_type

{0.0: 178, 1.0: 1, 2.0: 44, 3.0: 7, 4.0: 1}

In [3]:
uid_list = torch.load(negative_uid)
save_path = '/hdd/sd8/tlc/TCT/all_data/500_neg_416/'
for uid in tqdm_notebook(uid_list):
    new_dir = os.path.join(save_path, uid)
    folder_path = os.path.join(data_path_500, uid)
    os.mkdir(new_dir)
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        image = cv2.imread(img_path)
        save_dir = os.path.join(new_dir, img)
        image = img_resize(image, (416,416))
        cv2.imwrite(save_dir, image)